In [ ]:
from bs4 import BeautifulSoup
import csv
import os
import shutil
import time

## --- parameter ---
CONTEST = "BW132" # Contest name
CONTEST_URL = "https://leetcode.com/contest/biweekly-contest-132/" # Contest URL
LANG = ".cpp" # the file extension gonna deal with
OUTPUT_HTML = CONTEST+"/Q4_cpp.html" # the output HTML generated by the detection tool
THRESHOLD = 90 # Merge only if Both A is THRESHOLD% similar to B, and B is THRESHOLD% similar to B

In [ ]:
def Manual_add(user1, user2, cheater):
    group_1 = -1;
    group_2 = -1;
    
    for j, group in enumerate(cheater):
        if user1 in group:
            group_1 = j
        if user2 in group:
            group_2 = j
    
    if group_1 == -1 and group_2 == -1:
        cheater.append([user1, user2])
    elif group_1 == -1:
        cheater[group_2].append(user1)
    elif group_2 == -1:
        cheater[group_1].append(user2)
    elif group_1 != group_2:
        cheater[group_1].extend(cheater[group_2])
        del cheater[group_2]

In [ ]:
with open(OUTPUT_HTML, encoding="utf-8") as f:
    start = time.time()
    soup = BeautifulSoup(f.read(), 'html.parser')
    end = time.time()
    print("time elapsed:", end - start)

In [ ]:
## --- parameter ---
# The cheater group found so far

cheater = [
]

In [ ]:
# It will output the cheater list. You can copy the result to the code block above to record it

## --- parameter ---
IGNORE = [] # user to ignore, i.e. maybe you know they're not cheating or exclude temporary
Manual_add(739, 902, cheater) # Manually merge cheater

for row in soup.find_all("tr"):
    i = row.find_all("i")

    user1 = int(i[0].get_text()[13:-len(LANG)])
    user2 = int(i[1].get_text()[13:-len(LANG)])

    if user1 in IGNORE or user2 in IGNORE:
        continue

    b = row.find_all("b")
    p1 = float(b[0].get_text()[:-1])
    p2 = float(b[1].get_text()[:-1])

    group_1 = -1;
    group_2 = -2;
    
    for j, group in enumerate(cheater):
        if user1 in group:
            group_1 = j
        if user2 in group:
            group_2 = j

    if (user1 == -1 or user2 == -1) and (p1 >= THRESHOLD or p2 >= THRESHOLD):
        if group_1 == -1 and group_2 == -2:
            cheater.append([user1, user2])
        elif group_1 == -1:
            cheater[group_2].append(user1)
        elif group_2 == -2:
            cheater[group_1].append(user2)
        elif group_1 != group_2:
            cheater[group_1].extend(cheater[group_2])
            del cheater[group_2]
        

    if p1 >= THRESHOLD and p2 >= THRESHOLD:
        if group_1 == -1 and group_2 == -2:
            cheater.append([user1, user2])
        elif group_1 == -1:
            cheater[group_2].append(user1)
        elif group_2 == -2:
            cheater[group_1].append(user2)
        elif group_1 != group_2:
            cheater[group_1].extend(cheater[group_2])
            del cheater[group_2]
    elif group_1 != group_2:
        print(f"Test file: {i[0].get_text()} ({b[0].get_text()})\nReference file: {i[1].get_text()} ({b[1].get_text()})")

print("[")
for i in cheater:
    i.sort()
    print(i, ",")
print("]")

In [ ]:
# # clear 100%
# QUESTION = '/Q3/'

# for group in cheater:
#     for idx in range(1, len(group)):
#         folder = str(group[idx]//1000)+"001-"+str(group[idx]//1000+1)+"000/"
        
#         if os.path.exists(CONTEST+QUESTION+folder+str(group[idx])+LANG):
#             os.remove(CONTEST+QUESTION+folder+str(group[idx])+LANG)

In [ ]:
# Not necessary. Copy to test for further testing the first file of each group
QUESTION = '/Q4/'

try:
    shutil.rmtree(CONTEST+"/test")
except:
    pass
try:
    os.mkdir(CONTEST+"/test")
except:
    pass

for group in cheater:
    folder = str(group[0]//1000)+"001-"+str(group[0]//1000+1)+"000/"

    shutil.copyfile(CONTEST+QUESTION+folder+str(group[0])+LANG, CONTEST+"/test/"+str(group[0])+LANG)

In [ ]:
## generate markdown, and record the rank, user name, and submission ID of the mass cheating group to csv

## --- parameter ---
Q = 4 # i.e. 4 means Q4
CSV = CONTEST+"/Q4_cpp.csv" # the name of csv gonna be created to record the mass cheating group
# the mass cheating group
cheater = [234, 251, 252, 258, 261, 273, 276, 282, 286, 289, 291, 295, 302, 308, 310, 319, 326, 329, 331, 333, 338, 339, 343, 348, 349, 350, 351, 356, 358, 360, 362, 365, 368, 374, 376, 377, 378, 382, 395, 399, 404, 405, 410, 414, 415, 416, 423, 424, 428, 429, 432, 433, 435, 436, 437, 442, 445, 447, 448, 450, 452, 457, 461, 462, 463, 469, 470, 471, 473, 478, 479, 486, 489, 491, 493, 504, 509, 519, 526, 528, 536, 537, 540, 544, 546, 549, 550, 552, 553, 556, 558, 562, 564, 567, 569, 571, 573, 578, 580, 581, 585, 588, 591, 593, 597, 601, 602, 603, 605, 606, 608, 609, 612, 614, 616, 618, 619, 623, 625, 629, 636, 639, 640, 643, 648, 649, 650, 652, 654, 655, 656, 657, 661, 663, 664, 665, 667, 668, 669, 671, 672, 673, 677, 680, 684, 688, 689, 691, 694, 697, 698, 703, 707, 715, 716, 720, 723, 725, 732, 734, 736, 741, 742, 746, 747, 750, 752, 755, 760, 763, 764, 769, 770, 772, 773, 776, 780, 784, 786, 789, 792, 799, 801, 802, 813, 815, 822, 823, 824, 827, 828, 829, 832, 835, 837, 841, 845, 848, 852, 854, 860, 861, 862, 864, 866, 871, 872, 873, 875, 880, 886, 887, 889, 892, 896, 897, 899, 900, 901, 909, 911, 914, 915, 916, 919, 920, 921, 923, 924, 926, 929, 930, 931, 934, 935, 938, 940, 941, 943, 944, 955, 958, 959, 960, 962, 968, 969, 970, 971, 4587, 5212, 5662, 5677, 5678, 5679]

print("how many cheater: ", len(cheater))

with open(CONTEST+"/"+CONTEST+".csv") as f:
    rows = list(csv.reader(f))
    
    with open(CSV, "w") as csv_:
        for i in cheater:
            print(str(i)+"," + rows[i][0]+"," + rows[i][Q], file=csv_)

    with open(CSV[:-4]+".md", "w") as md:
        print("| rank | user | Q"+str(Q)+"   |\n| ---- | ---- | ---- |", file=md)
    
        for i in cheater:
            print("|", i, "| ["+rows[i][0]+"](https://leetcode.com/u/"+rows[i][0]+") |", CONTEST_URL+"submissions/detail/"+rows[i][Q], "|", file=md)